In [ ]:
!pip install efficientnet_pytorch
!pip install onnx_tf
!pip install onnxruntime
!pip install onnx_tf

import cv2
import torch
from torch import nn
import efficientnet_pytorch
from matplotlib import pyplot as plt

In [ ]:
path_to_model = '../input/eff-net/efficientnet.torch'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = efficientnet_pytorch.EfficientNet.from_pretrained("efficientnet-b0")
model._fc = nn.Linear(in_features=1280, out_features=5, bias=True)
model.to(device)
model.load_state_dict(
            torch.load(path_to_model, map_location=device)["model_state_dict"]
)
model.eval();

In [ ]:
from albumentations import Normalize, Compose, Resize
from albumentations.pytorch import ToTensorV2

IMAGE_SIZE = (256, 256)

transforms_valid = Compose(
    [
        Resize(*IMAGE_SIZE),
        Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
        ToTensorV2(),
    ],
    p=1,
)

In [ ]:
def predict(img, model, device, transform):
    img = transform(image=img)["image"]
    img = torch.unsqueeze(img, 0)
    x_test = img.to(device)

    with torch.no_grad():
        _, pred = model(x_test).max(1)
    predict = pred.tolist()
    return predict[0]

In [ ]:
image = cv2.imread('../input/cassava-leaf-disease-classification/train_images/100533489.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image);
plt.title(predict(image, model, device, transforms_valid));

In [ ]:
img = transforms_valid(image=image)["image"]
img = torch.unsqueeze(img, 0)
dummy_input = img
model(dummy_input).max(1)

In [ ]:
model.set_swish(memory_efficient=False) # https://github.com/lukemelas/EfficientNet-PyTorch/issues/91

# Export Model

In [ ]:
path_to_save_model = './EffNet.onnx'

torch.onnx.export(model,         # model being run 
     dummy_input.to(device),     # model input (or a tuple for multiple inputs) 
     path_to_save_model,         # where to save the model  
     export_params = True,       # store the trained parameter weights inside the model file 
     do_constant_folding = True, # whether to execute constant folding for optimization
     input_names = ['input'],    # the model's input names
     output_names = ['output'])  # the model's output names

print('Model has been converted to ONNX')

# Load the model using onnx.load

In [ ]:
import onnx

onnx_model = onnx.load(path_to_save_model)
onnx.checker.check_model(onnx_model)

# Create inference session with onnxruntime.infernnce

In [ ]:
import onnxruntime as onnxrt

# Lauch the session
onnx_session = onnxrt.InferenceSession(path_to_save_model)

# Assigns the input numpy array as input vector for the onnx session
onnx_inputs = {onnx_session.get_inputs()[0].name: dummy_input.numpy()}

# Inference
onnx_predictions = onnx_session.run(None, onnx_inputs)
onnx_predictions[0].argmax(1)[0]

In [ ]:
from onnx_tf.backend import prepare  # The "prepare" function is a wrapper for a session in tensorflow

# Load the onnx model with onnx module
onnx_model = onnx.load(path_to_save_model)

# Inference
tensorflow_predictions = prepare(onnx_model).run(dummy_input.numpy())[0]; 
print(tensorflow_predictions.argmax(1)[0])

# https://www.canva.com/design/DAE2veRgTBE/6ytSoSs5eqTHcb6a8hDEbg/view?utm_content=DAE2veRgTBE&utm_campaign=designshare&utm_medium=link&utm_source=publishsharelink